# W266 Project Milestone Code
Marcus DeMaster

### **Collect Labeled Subjects and Tweet Histories**

In [3]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import tweepy
import json

#Twitter API Credentials for using tweepy
consumer_key = "zX6qyiIyZze3gu5r1We9IUgqY"
consumer_secret = "NeU4ossMcQziXIiAqc6xxLwiojdnwIhFrgeCuJlWoInrHyKmxM";
access_token = "837785154651238400-zVPiieFeIFH7LQq4XGFPuj2UGNsRXZS";
access_token_secret = "9EwL5ov6Ej99zpUPkyKCdOUMVnov8paUTPpgWYcsbBigF";
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)  

#Set empty list to append json objects
tjson=[]

#Set search parameters for model subject tweets to be webscraped.
start='2017-09-01'
end='2017-11-17'
querylist=["I have health insurance","I don't have health insurance"]

#Number of tweets from user history to collect
tweethistcount=50

#Query tweets stating they do and do not have health insurance.
for label,query in enumerate(querylist):
    print
    print 'Collecting "%s" tweets from %s to %s and a tweet-history of %s past tweets'%(query,start,end,tweethistcount)
    print
    startTime = datetime.now()
    q=query.replace(' ','%20')
    prefix='https://twitter.com/search?f=tweets&vertical=default&q=%22'
    browser = webdriver.Chrome('./chromedriver')
    browser.get(prefix+q+'%22%20since%3A'+start+'%20until%3A'+end+'&src=typd')
    
    #Scroll to bottom of search results page to webscrape all results.
    while True:
        elemsCount = browser.execute_script("return document.querySelectorAll('.stream-items > li.stream-item').length")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        try:
            WebDriverWait(browser, 2).until(lambda x: x.find_element_by_xpath(
                "//*[contains(@class,'stream-items')]/li[contains(@class,'stream-item')]["+str(elemsCount+1)+"]"))
        except:
            break  
    
    #Get lists of all tweets, usernames, and dates
    tweets = browser.find_elements_by_class_name('js-tweet-text')
    tweetids = browser.find_elements_by_css_selector('li.js-stream-item')
    users = browser.find_elements_by_css_selector('.original-tweet')
    dates = browser.find_elements_by_class_name('tweet-timestamp')
    
    twtcnt=0
    
    #Loop through tweet info and assign to dict
    for user,date,tweetid,tweet in zip(users,dates,tweetids,tweets):
        tdata={}
        tdata['tweet_id']=tweetid.find_element_by_css_selector('.time a.tweet-timestamp').get_attribute('data-conversation-id')
        tdata['label']=label
        tdata['tweetdate']=date.get_attribute('title')
        tdata['username']=user.get_attribute('data-screen-name')
        tdata['tweettext']=tweet.text.encode("utf-8")
#         \
#         .decode('unicode_escape')\
#         .encode('ascii','ignore')\
#         .replace('"', '""')\
#         .replace('\n', ' ')\
#         .replace('\r', ' ')\
#         .replace('  ', ' ')\
#         .replace(',',' ')
        
        #Ignore tweets in quotes and from bots
        T=tdata['tweettext'].upper()
        Q=query.upper()
        #all(['"' in T.split(Q,1)[0], '"' in T.split(Q,1)[1]]) or 
        if any(['bot' in tdata['username'],'censusAmericans'in tdata['username']]):
            continue
        else:
            twtcnt+=1
            pasttweets=[]
            pasttweetdates=[]
            #For eligible subjects, use API to search prior user timeline tweets and add to dict
            for status in tweepy.Cursor(
                    api.user_timeline,
                    tdata['username'],
                    max_id=int(tdata['tweet_id'])-1
                    ).items(tweethistcount):
                pasttweet=status.text.encode("utf-8")#\
                #.decode('unicode_escape')\
                #.encode('ascii','ignore')\
                #.replace('\n', ' ')\
                #.replace('\r', ' ')\
                #.replace('  ', ' ')\
                #.replace(',',' ')
                pasttweets.append(pasttweet)
                pasttweetdates.append(str(status.created_at))
            tdata['pasttweets']=pasttweets
            tdata['pasttweetdates']=pasttweetdates
            tjson.append(tdata)    
        if twtcnt % 100 == 0:
            print '-----%s tweets collected at %s.'%(twtcnt,datetime.now())
    browser.quit()
    print '-----%s "%s" tweets collected in %s seconds.'%(twtcnt,query,datetime.now()-startTime)

#Write dicts to json file.
with open('testtweets.json', 'w') as outfile:
    outfile.write(json.dumps(tjson))



-----100 tweets collected at 2017-11-17 20:28:07.611147.
-----200 tweets collected at 2017-11-17 20:29:41.832015.
-----300 tweets collected at 2017-11-17 20:31:14.246511.
-----400 tweets collected at 2017-11-17 20:32:45.796151.
-----500 tweets collected at 2017-11-17 20:44:35.886277.
-----600 tweets collected at 2017-11-17 20:46:10.498302.
-----700 tweets collected at 2017-11-17 20:47:35.400911.
-----741 "I have health insurance" tweets collected in 0:22:19.339171 seconds.


-----100 tweets collected at 2017-11-17 20:59:57.658332.
-----200 tweets collected at 2017-11-17 21:01:24.124802.
-----300 tweets collected at 2017-11-17 21:02:59.551815.
-----400 tweets collected at 2017-11-17 21:14:35.203064.
-----500 tweets collected at 2017-11-17 21:16:08.609713.
-----600 tweets collected at 2017-11-17 21:17:38.573992.
-----631 "I don't have health insurance" tweets collected in 0:29:54.572678 seconds.


### Exploratory Analysis

In [5]:
#Read json with pandas and inspect
import pandas as pd
d = pd.read_json('testtweets.json')
d.head()

,label,pasttweetdates,pasttweets,tweet_id,tweetdate,tweettext,username
0,0,"[2017-11-16 22:52:05, 2017-11-16 22:49:26, 201...","[I'm tirrrrrred, @HarrisHarrisev9 Such truth!!...",931295366527553536,2:58 PM - 16 Nov 2017,I have health insurance now so that's good,thatwitchlefay
1,0,"[2017-11-16 21:27:45, 2017-11-16 21:18:49, 201...",[RT @nytopinion: The House just passed a bill ...,931272809594093568,2:15 PM - 16 Nov 2017,I teach one section of first-year writing each...,the_author_
2,0,"[2017-11-16 21:53:13, 2017-11-16 21:51:46, 201...",[RT @illscum: honestly at this point the probl...,931279638734757888,1:55 PM - 16 Nov 2017,I literally take a medication for my addiction...,37notmywife
3,0,"[2017-11-16 16:03:04, 2017-11-16 05:55:37, 201...",[@TomJurgensen @sirdinitski I literally just g...,931202725366386688,8:50 AM - 16 Nov 2017,I have health insurance again so I’m bout to b...,spotifyisokay
4,0,"[2017-11-16 02:01:12, 2017-11-16 01:55:40, 201...",[RT @WayneDupreeShow: I AM NOT A COLOR. I am a...,931065177906216960,8:10 AM - 16 Nov 2017,"They will not lose healthcare, they will lose ...",Sheeple15


In [6]:
#Inspect label balance
d.groupby('label')['tweet_id'].count()

label
0    741
1    631
Name: tweet_id, dtype: int64

In [7]:
#Inspect distribution of subjects by prior tweet history volumes
d.groupby(d['pasttweets'].str.len())['tweet_id'].count()

pasttweets
0      127
1        2
2        1
4        2
5        1
9        1
11       2
12       1
15       1
16       1
17       1
18       2
19       2
20       1
21       3
23       1
25       2
28       1
30       1
32       2
33       1
40       1
41       4
42       1
45       1
47       1
50    1208
Name: tweet_id, dtype: int64

### Baseline Logistic Regression Classifier

In [8]:
#Convert list of past tweets to text for modeling.
d['pasttweets_text']=d['pasttweets'].apply(lambda x: ', '.join(x))

In [13]:
#Split subjects into train and test sets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
train_tweets, dev_tweets = train_test_split(d, test_size=0.2)

In [46]:
#Create TFIDF vectors for modeling
vectorizer = TfidfVectorizer(max_df=0.1)  
train_tweets_vector = vectorizer.fit_transform(train_tweets['pasttweets_text'])
dev_tweets_vector = vectorizer.transform(dev_tweets['pasttweets_text'])

In [47]:
#Logistic Regression Scoring Function
def PerformLogisticRegression(c, train_data, train_labels, dev_data, dev_labels):
    model = LogisticRegression(C=c ,class_weight='balanced')
    model.fit(train_data, train_labels)   
    predicted_labels = model.predict(dev_data)
    
    #scores
    score = metrics.f1_score(dev_labels,predicted_labels, pos_label = 1)
    f1a, f1b =metrics.f1_score(dev_labels,predicted_labels, average=None)
    precision = metrics.precision_score(dev_labels,predicted_labels, pos_label = 1)
    accuracy = np.mean(predicted_labels == dev_labels) 
    
    #roc_auc
    predicted_prob = model.predict_proba(dev_data) 
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels, predicted_prob[:,1], pos_label = 1)
    roc_auc = metrics.auc(fpr, tpr)
    print ' C: %3.5f ,  accuracy: %3.5f , precision-score:%3.5f,  f1-score: %3.5f, (%3.5f,%3.5f)  roc_auc: %3.5f ' %(c,  accuracy,precision,score , f1a, f1b, roc_auc )
    return (score, precision, accuracy,  model)

In [48]:
# Scan for the best C value 
c_values =  [ 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0 ]
max_score = 0;
max_set =()
for c in c_values:
    score, precision, accuracy, model = PerformLogisticRegression(
        c, train_tweets_vector, train_tweets['label'], dev_tweets_vector, dev_tweets['label'])
    
    if (score > max_score):
        max_score = score
        max_set = (c,accuracy, score, precision, model)

 C: 0.00010 ,  accuracy: 0.56727 , precision-score:0.53600,  f1-score: 0.52964, (0.59933,0.52964)  roc_auc: 0.59439 
 C: 0.00100 ,  accuracy: 0.57091 , precision-score:0.53968,  f1-score: 0.53543, (0.60135,0.53543)  roc_auc: 0.59439 
 C: 0.01000 ,  accuracy: 0.57818 , precision-score:0.54545,  f1-score: 0.55385, (0.60000,0.55385)  roc_auc: 0.59455 
 C: 0.10000 ,  accuracy: 0.58545 , precision-score:0.55072,  f1-score: 0.57143, (0.59859,0.57143)  roc_auc: 0.59683 
 C: 0.50000 ,  accuracy: 0.58909 , precision-score:0.55474,  f1-score: 0.57358, (0.60351,0.57358)  roc_auc: 0.59662 
 C: 1.00000 ,  accuracy: 0.58182 , precision-score:0.54815,  f1-score: 0.56274, (0.59930,0.56274)  roc_auc: 0.59673 
 C: 5.00000 ,  accuracy: 0.56727 , precision-score:0.53600,  f1-score: 0.52964, (0.59933,0.52964)  roc_auc: 0.59327 
 C: 10.00000 ,  accuracy: 0.57455 , precision-score:0.54472,  f1-score: 0.53386, (0.60870,0.53386)  roc_auc: 0.59035 
